In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import torch.nn as nn
import torch
import sklearn.preprocessing as preprocessing
import os

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader


import warnings
warnings.filterwarnings(action='ignore')
from copy import deepcopy

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution

In [2]:
# train.py

import os
import torch
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from data_loader import load_data_1m
from feature_calculations import (
    resample_data, calculate_MA_data, calculate_ema_bollinger_bands, calculate_rsi,
    calculate_macd, calculate_stochastic_oscillator, calculate_adx, calculate_atr,
    calculate_obv, calculate_williams_r, base_feature_fn, cyclic_encode_fn, log_transform
)
from strategies import BB_fitness_fn
from dataset import make_dataset, replace_nan_with_zero
from train_functions_bi_cul import inference, fitness_fn, generation_valid, generation_test, calculate_fitness

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution




In [3]:
state_dict_path = '/root/daily/bit_3/generation/generation_10.pt'
if os.path.exists(state_dict_path):
    state_dict = torch.load(state_dict_path)
    start_gen_idx = state_dict['generation'] + 1
    best_profit = state_dict['best_profit']
    best_chromosomes = state_dict['best_chromosomes']
    # prescriptor.load_state_dict(state_dict['prescriptor_state_dict'],strict=True)

In [5]:
best_profit

tensor([], size=(0, 5))

In [5]:
best_profit[:, 4].max()

tensor(19.6312)

In [6]:
optimal_solution = best_profit[(best_profit[:, 4] > 0.8) & (best_profit[:, 3] <= 40)]
optimal_index = torch.where((best_profit[:, 4] > 0.8) & (best_profit[:, 3] <= 40))[0]

In [7]:
best_profit[optimal_index][:, 4]

tensor([ 0.8700,  1.0388,  0.8606,  1.8945,  1.2377,  0.8272,  1.1439,  1.5217,
         0.9918,  1.6967,  0.8210,  1.1516,  0.9556,  0.8096,  1.0517,  0.9396,
        19.6312,  0.8853,  0.9163,  0.8953,  0.8664,  1.3702,  0.9550,  0.9357,
         1.1248,  1.2498,  1.2572,  1.4123,  0.8172,  1.6610,  0.8916,  0.8825,
         1.4897,  0.8936,  0.9957,  0.9639,  0.8127,  2.9503,  0.9252,  0.9528,
         2.4446,  0.8613,  0.9465,  1.2260,  0.8132,  1.2418,  0.8103,  1.9995,
         2.8555,  1.3655,  1.6267,  0.9516,  0.9256,  1.0595,  0.9540,  1.3871,
         0.8363,  0.8468,  0.8780,  0.8816,  1.0679,  0.9977,  1.0511,  1.1510,
         0.8774,  0.8752,  0.9810,  0.8813,  1.0727,  1.1453,  1.1805,  0.9143,
         2.3963])

In [8]:
fitness = calculate_fitness(deepcopy(optimal_solution.numpy()))
fitness_index = np.argsort(-fitness)

In [9]:
index = 3

In [28]:
optimal_solution[fitness_index][:, 0][index]

tensor(0.0292)

In [29]:
optimal_solution[fitness_index][:, 1][index]

tensor(1.2296)

In [30]:
optimal_solution[fitness_index][:, 2][index]

tensor(0.5037)

In [31]:
optimal_solution[fitness_index][:, 3][index]

tensor(2.9445)

In [32]:
optimal_solution[fitness_index][:, 4][index]

tensor(1.0388)